# Pathology prediction (fracture)

@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the fracture pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples :  0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value. 

The model is based on binary classification for each target chest pathology.

Final sample size for the fracture pathology is : (N = 557)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.FRACTURE, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }
# Fixed parameters to pass
fixed_params = {
    'seed': 42,
    'eval_metric': 'logloss',
    'verbosity': 0
}

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Fracture", experiment_name="CP_Fracture", filepath="./results/fracture")

evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='kfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps,  # Grid d'hyperparamètres
    fixed_params=fixed_params # Paramètres fixes
)

2024-09-24 15:22:48,738	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


(run_fold pid=95326) Outer fold 1
(run_fold pid=95324) Configuring PyCaret for outer fold 4
(run_fold pid=95336) Outer fold 3 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Processing:  75%|███████▌  | 3/4 [00:22<00:08,  8.20s/it] [repeated 5x across cluster]


(run_fold pid=95324)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=95324) Fold                                                          
(run_fold pid=95324) 0       0.9722  0.8164     1.0  0.9718  0.9857  0.4894  0.5692
(run_fold pid=95324) 1       0.9437  0.7333     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=95324) 2       0.9577  0.8030     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=95324) 3       0.9296  0.8000     1.0  0.9296  0.9635  0.0000  0.0000
(run_fold pid=95324) 4       0.9437  0.9818     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=95324) Mean    0.9494  0.8269     1.0  0.9487  0.9736  0.3355  0.4112
(run_fold pid=95324) Std     0.0145  0.0827     0.0  0.0144  0.0075  0.1920  0.2182
(run_fold pid=95336) Configuring PyCaret for outer fold 3 [repeated 4x across cluster]


(run_fold pid=95324) Tuning hyperparameters for model xgboost with custom grid using grid search


Processing:  75%|███████▌  | 3/4 [00:23<00:08,  8.45s/it] [repeated 4x across cluster]


(run_fold pid=95336) 0       0.9583  0.7391     1.0  0.9583  0.9787  0.0000  0.0000
(run_fold pid=95336) 1       0.9577  0.6667     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=95336) 2       0.9437  0.7909     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=95336) 3       0.9437  0.6424     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=95336) 4       0.9577  0.7788     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=95329) 0       0.9722  0.8744     1.0  0.9718  0.9857  0.4894  0.5692
(run_fold pid=95329) 1       0.9437  0.8909     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=95329) 2       0.9296  0.7212     1.0  0.9296  0.9635  0.0000  0.0000
(run_fold pid=95329) 3       0.9577  0.7091     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=95329) 4       0.9437  0.7424     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=95326) 0       0.9722  0.8551     1.0  0.9718  0.9857  0.4894  0.5692
(run_fold pid=95326) 1       0.9437  0.7030     1.0  0.9429  0.9706  0.3173 

                                                          [repeated 4x across cluster]
